In [126]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

In [127]:
import pandas as pd

## Вставление url

In [162]:
url  = 'https://www.goszakup.gov.kz/ru/registry/rqc?count_record=50&page=10'

In [163]:
data  = requests.get(url,verify=False).text
soup = BeautifulSoup(data,"html.parser")
tables = soup.find_all('table')

In [164]:
for tables in soup.find_all('table'):
    print(tables.text)




№
Наименование потенциального поставщика
БИН/ИИН
Наименование, номер и дата выдачи документа, на основании которого потенциальный поставщик включен в Перечень




538


Филиал "Еңбек-Караганда" РГП на ПХВ "Еңбек" учреждений уголовно-исполнительной (пенитенциарной) системы Комитета уголовно-исполнительной системы МВД РК


180441017316
110000017, 2099-12-31 00:00:00


386


Товарищество с ограниченной ответственностью "Обувная фабрика SAMHAT"


140340011221
103900001, 2019-08-01 00:00:00


858


Товарищество с ограниченной ответственностью "SKINWOOD"


180740028654
101100047, 2099-12-31 00:00:00


560


ТОО "Arta Software"


050940002848
101900059, 2099-12-31 00:00:00


873


ТОО "MSSP.GL"


150140024604
101000030, 2099-12-31 00:00:00


561


ТОО "KAZPROTECT" (КАЗПРОТЕКТ)


140440021222
110000027, 2099-12-31 00:00:00


839


ТОО "PDMnet"


110740011449
101100035, 2099-12-31 00:00:00


798


Товарищество с ограниченной ответственностью "Өнерпаз"


020940002804
106100003, 2099-12-31 00:

## Парсинг Наименований и БИН

In [165]:
df1 = pd.DataFrame(columns=["№", "Наименование потенциального поставщика", "БИН/ИИН", 
                                        "Наименование, номер и дата выдачи документа"])

for row in tables.tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        num = col[0].text
        name = col[1].text
        bin_iin = col[2].text.strip()
        nomer = col[3].text.strip()
        df1 = df1.append({"№":num, "Наименование потенциального поставщика":name , 
                                                  "БИН/ИИН":bin_iin, "Наименование, номер и дата выдачи документа":nomer}, ignore_index=True)

df1.head()

,№,Наименование потенциального поставщика,БИН/ИИН,"Наименование, номер и дата выдачи документа"
0,538,"\n\nФилиал ""Еңбек-Караганда"" РГП на ПХВ ""Еңбек...",180441017316,"110000017, 2099-12-31 00:00:00"
1,386,\n\nТоварищество с ограниченной ответственност...,140340011221,"103900001, 2019-08-01 00:00:00"
2,858,\n\nТоварищество с ограниченной ответственност...,180740028654,"101100047, 2099-12-31 00:00:00"
3,560,"\n\nТОО ""Arta Software""\n\n",050940002848,"101900059, 2099-12-31 00:00:00"
4,873,"\n\nТОО ""MSSP.GL""\n\n",150140024604,"101000030, 2099-12-31 00:00:00"


## Получение ссылок с каждой наименовании для изъятия ФИО,ИИН,АДРЕС

In [166]:
urls = [tag.find("a")["href"] for tag in soup.select("td:has(a)")]

## Получение ИИН, ФИО

In [167]:
import urllib3
urllib3.disable_warnings()
session = requests.Session()
session.trust_env = False
iin = []
name = []
for url in urls:
    data1  = requests.get(url,verify=False).text
    soup = BeautifulSoup(data1,"html.parser")
    tables = soup.find_all('table')
    v_data = pd.DataFrame(columns=["Значения"])
    for row in tables[2].find_all("tr"):
        col = row.find_all("td")
        if (col != []):
            value = col[0].text
            v_data = v_data.append({"Значения":value}, ignore_index=True)
    iin.append(v_data['Значения'][0])
    name.append(v_data['Значения'][2])

In [168]:
df1['ИИН Руководителя'] = iin
df1['ФИО'] = name

In [169]:
df1.head()

,№,Наименование потенциального поставщика,БИН/ИИН,"Наименование, номер и дата выдачи документа",ИИН Руководителя,ФИО
0,538,"\n\nФилиал ""Еңбек-Караганда"" РГП на ПХВ ""Еңбек...",180441017316,"110000017, 2099-12-31 00:00:00",690731300978,КУЛХАЖИЕВ АСЕТ ШЕКЕРБЕКОВИЧ
1,386,\n\nТоварищество с ограниченной ответственност...,140340011221,"103900001, 2019-08-01 00:00:00",890621350056,ИСАБЕКОВ АСХАТ ЖУМАБЕКОВИЧ
2,858,\n\nТоварищество с ограниченной ответственност...,180740028654,"101100047, 2099-12-31 00:00:00",860717350974,АХМЕТОВ АДИЛЬХАН ДЖАНБУЛАТОВИЧ
3,560,"\n\nТОО ""Arta Software""\n\n",050940002848,"101900059, 2099-12-31 00:00:00",830502350038,АЛПЫСПАЕВ БАУРЖАН КОШУРБАЕВИЧ
4,873,"\n\nТОО ""MSSP.GL""\n\n",150140024604,"101000030, 2099-12-31 00:00:00",901006350179,САЛИПОВ ДАУРЕН КАИРГЕЛЬДИНОВИЧ


## Получения Адресов

In [170]:
import urllib3
urllib3.disable_warnings()
address = []
for url in urls:
    data1  = requests.get(url,verify=False).text
    soup = BeautifulSoup(data1,"html.parser")
    tables = soup.find_all('table')
    v_data = pd.DataFrame(columns=["Адрес"])
    for row in tables[3].find_all("tr"):
        col = row.find_all("td")
        if (col != []):
            rank = col[2].text
            v_data = v_data.append({"Адрес":rank}, ignore_index=True)
    address.append(v_data['Адрес'][0].strip())

In [171]:
len(address)

9

In [172]:
df1['Адрес'] = address

In [173]:
df1.head()

,№,Наименование потенциального поставщика,БИН/ИИН,"Наименование, номер и дата выдачи документа",ИИН Руководителя,ФИО,Адрес
0,538,"\n\nФилиал ""Еңбек-Караганда"" РГП на ПХВ ""Еңбек...",180441017316,"110000017, 2099-12-31 00:00:00",690731300978,КУЛХАЖИЕВ АСЕТ ШЕКЕРБЕКОВИЧ,"Карагандинская область, г.Караганда, Поспелова..."
1,386,\n\nТоварищество с ограниченной ответственност...,140340011221,"103900001, 2019-08-01 00:00:00",890621350056,ИСАБЕКОВ АСХАТ ЖУМАБЕКОВИЧ,"Акмолинская область, Целиноградский район, с.А..."
2,858,\n\nТоварищество с ограниченной ответственност...,180740028654,"101100047, 2099-12-31 00:00:00",860717350974,АХМЕТОВ АДИЛЬХАН ДЖАНБУЛАТОВИЧ,"г.Нур-Султан, Бейбитшилик, 54А"
3,560,"\n\nТОО ""Arta Software""\n\n",050940002848,"101900059, 2099-12-31 00:00:00",830502350038,АЛПЫСПАЕВ БАУРЖАН КОШУРБАЕВИЧ,"г.Нур-Султан, район Есиль, проспект Мангилик Е..."
4,873,"\n\nТОО ""MSSP.GL""\n\n",150140024604,"101000030, 2099-12-31 00:00:00",901006350179,САЛИПОВ ДАУРЕН КАИРГЕЛЬДИНОВИЧ,"г.Нур-Султан, Туркистан, 14А, 134"


## Очистка данных

In [174]:
df1['Наименование потенциального поставщика'] = df1['Наименование потенциального поставщика'].apply(lambda x : x.strip())

In [175]:
df1.drop(columns =['Наименование, номер и дата выдачи документа'],inplace=True)

In [176]:
df1.head()

,№,Наименование потенциального поставщика,БИН/ИИН,ИИН Руководителя,ФИО,Адрес
0,538,"Филиал ""Еңбек-Караганда"" РГП на ПХВ ""Еңбек"" уч...",180441017316,690731300978,КУЛХАЖИЕВ АСЕТ ШЕКЕРБЕКОВИЧ,"Карагандинская область, г.Караганда, Поспелова..."
1,386,"Товарищество с ограниченной ответственностью ""...",140340011221,890621350056,ИСАБЕКОВ АСХАТ ЖУМАБЕКОВИЧ,"Акмолинская область, Целиноградский район, с.А..."
2,858,"Товарищество с ограниченной ответственностью ""...",180740028654,860717350974,АХМЕТОВ АДИЛЬХАН ДЖАНБУЛАТОВИЧ,"г.Нур-Султан, Бейбитшилик, 54А"
3,560,"ТОО ""Arta Software""",050940002848,830502350038,АЛПЫСПАЕВ БАУРЖАН КОШУРБАЕВИЧ,"г.Нур-Султан, район Есиль, проспект Мангилик Е..."
4,873,"ТОО ""MSSP.GL""",150140024604,901006350179,САЛИПОВ ДАУРЕН КАИРГЕЛЬДИНОВИЧ,"г.Нур-Султан, Туркистан, 14А, 134"


## И повторяем эти шаги для всех 10 страниц, и сохраняем их

In [177]:
df1.to_excel(r'Desktop\tenth.xlsx')

## Полный датасет смотрите в файле Join_Dataset.ipynb, там найдете полный датасет 